In [5]:
import autogen
from dotenv import load_dotenv

load_dotenv()

config_list_file_name = ".config_list"
gemini_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gemini"]})

code_exec_config = {
    "last_n_messages": 3,
    "work_dir": "working_dir_proj",
    "use_docker": True  # Use Docker for isolation
}

user_proxy = autogen.UserProxyAgent(
    name="User_Agent",
    system_message="A human user who needs help from other agents to collaboratively generate a project on python and streamlit.",
    code_execution_config=code_exec_config,
    human_input_mode="NEVER"
)

coder = autogen.AssistantAgent(
    name="Expert_Coder_Agent",
    system_message="""You are a professional expert Python coder, debugger, and test code executor. 
    When Expert_Planner_Agent asks to generate code,
    Your tasks are to:
       1. Understand the requirements and architecture of the project.
       2. Create project folder and file hierarchy.
       3. Implement required files and run them to confirm that they execute without errors/exceptions.
       4. In case of any errors/exceptions, debug and fix them.
       5. Install required dependencies (e.g., Python modules using pip) to fix issues.
       6. In case of streamlit app, simply check bash logs if the runs successfully without any exception
          as there is no way to check the output for now. Also consider, allowing proxy in the port.
       7. Upon completion a successful run, respond to Expert_Planner_Agent with "TERMINATE".
    """,
    llm_config={"config_list": gemini_config_list, "timeout": 120},
    code_execution_config=code_exec_config
)

planner = autogen.AssistantAgent(
    name="Expert_Planner_Agent",
    system_message="""You are a professional expert in planning how to build a Streamlit and Python project. 
    You should take initial instructions from User_Agent and create a robust plan for the desired project.
    Your tasks are to:
       1. Understand the project's requirements.
       2. Create a robust plan, including the initial architecture, modules, related files, and their interrelationships.
       3. Instruct Expert_Coder_Agent to implement the related files and debug until the desired result is achieved.
       4. Upon completion, inform User_Agent that the task is completed.
    """,
    llm_config={"config_list": gemini_config_list, "timeout": 120},
    code_execution_config=code_exec_config
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, planner, coder],  # Include planner in the agents list
    messages=[],
    max_round=20,
    speaker_selection_method='auto',
    allow_repeat_speaker=True
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config={"config_list": gemini_config_list},
)

project_description = """A Streamlit app that will print "Hello World" on the main page and have a text input box in the sidebar."""
project_location = "sample_project"

user_proxy.initiate_chat(
    manager,
    message=f"""Your task is to generate a full Streamlit, Python project for the following project description: {project_description}.
    1. Collaborate with Expert_Planner_Agent and Expert_Coder_Agent to get the job done.
    2. Project code should be saved at {project_location}.
    """,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
)


User_Agent (to chat_manager):

Your task is to generate a full Streamlit, Python project for the following project description: A Streamlit app that will print "Hello World" on the main page and have a text input box in the sidebar..
    1. Collaborate with Expert_Planner_Agent and Expert_Coder_Agent to get the job done.
    2. Project code should be saved at sample_project.
    

--------------------------------------------------------------------------------

Next speaker: Expert_Planner_Agent

Expert_Planner_Agent (to chat_manager):

Okay, User_Agent. I understand your requirements.  We'll build a Streamlit app with a "Hello World" message on the main page and a text input box in the sidebar. The project will be saved in the `sample_project` directory.

Here's the plan I've devised for Expert_Coder_Agent:

**Project Name:** sample_project

**Project Structure:**

```
sample_project/
├── app.py          # Main Streamlit application file
└── requirements.txt # Project dependencies
```

ChatResult(chat_id=None, chat_history=[{'content': 'Your task is to generate a full Streamlit, Python project for the following project description: A Streamlit app that will print "Hello World" on the main page and have a text input box in the sidebar..\n    1. Collaborate with Expert_Planner_Agent and Expert_Coder_Agent to get the job done.\n    2. Project code should be saved at sample_project.\n    ', 'role': 'assistant', 'name': 'User_Agent'}, {'content': 'Okay, User_Agent. I understand your requirements.  We\'ll build a Streamlit app with a "Hello World" message on the main page and a text input box in the sidebar. The project will be saved in the `sample_project` directory.\n\nHere\'s the plan I\'ve devised for Expert_Coder_Agent:\n\n**Project Name:** sample_project\n\n**Project Structure:**\n\n```\nsample_project/\n├── app.py          # Main Streamlit application file\n└── requirements.txt # Project dependencies\n```\n\n**Modules and Files:**\n\n* **`app.py`**: This file will c